<a href="https://colab.research.google.com/github/mansi0411/Text2SQL-via-Prompt-Engineering/blob/main/Text2SQL_via_Prompt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! curl "https://api.mockaroo.com/api/b156bcd0?count=1000&key=0c6cf4e0" > "customers.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   99k    0   99k    0     0  61959      0 --:--:--  0:00:01 --:--:-- 61958


In [2]:
! curl "https://api.mockaroo.com/api/d7db2ad0?count=1000&key=0c6cf4e0" > "products.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  328k    0  328k    0     0   214k      0 --:--:--  0:00:01 --:--:--  214k


In [3]:
! curl "https://api.mockaroo.com/api/5193eb90?count=1000&key=0c6cf4e0" > "orders.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 79353    0 79353    0     0  18089      0 --:--:--  0:00:04 --:--:-- 18092


In [4]:
import sqlite3
import pandas as pd
import os

In [5]:
customers_schema="""
CREATE TABLE IF NOT EXISTS customers(
	customer_id INT PRIMARY KEY,
	first_name VARCHAR(50),
	last_name VARCHAR(50),
	email VARCHAR(50),
	phone_number VARCHAR(50),
	address VARCHAR(50),
	city VARCHAR(50),
	country VARCHAR(50),
	postal_code VARCHAR(50),
	membership_level VARCHAR(7)
);
"""

products_schema="""
CREATE TABLE IF NOT EXISTS products(
	product_id INT PRIMARY KEY,
	product_name VARCHAR(50),
	product_category VARCHAR(50),
	product_price DECIMAL(8,2),
	product_description TEXT,
	product_weight DECIMAL(4,1),
	product_color VARCHAR(5),
	product_brand VARCHAR(50),
	product_material VARCHAR(9),
	product_size VARCHAR(11),
	product_stock_quantity INT
);
"""

orders_schema="""
CREATE TABLE IF NOT EXISTS orders(
	order_id INT PRIMARY KEY,
	customer_id VARCHAR(50),
	product_id VARCHAR(50),
	quantity INT,
	unit_price DECIMAL(6,2),
	total_price DECIMAL(10,2),
	order_date DATE,
	shipping_address VARCHAR(50),
	payment_method VARCHAR(16),
	status VARCHAR(9),
  FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
  FOREIGN KEY (product_id) REFERENCES products(product_id)
);
"""

In [6]:
db_name='ecommerce.db'
if os.path.exists(db_name):
  os.remove(db_name)
  print(f"Removed existing database '{db_name}'.")


In [7]:
import sqlite3
import pandas as pd
import os


COLUMN_DATA_TYPES={
    'customers':{
        'customer_id': 'int64',
        'first_name': 'object',
        'last_name': 'object',
        'email': 'object',
        'phone_number' : 'object',
        'address' : 'object',
        'city' : 'object',
        'country' : 'object',
        'postal_code' : 'object',
        'membership_level':'object'
},
    'products':{
        'product_id' : 'int64',
	      'product_name': 'object',
	      'product_category' : 'object',
	      'product_price' :'float64',
	      'product_description': 'object',
      	'product_weight':'float64',
	      'product_color' : 'object',
	      'product_brand': 'object',
	      'product_material' : 'object',
	      'product_size' : 'object',
	      'product_stock_quantity': 'int64'
    },

    'orders':{
        'order_id':'int64',
	      'customer_id': 'int64',
	      'product_id': 'int64',
	      'quantity': 'int64',
	      'unit_price' : 'float64',
        'total_price' : 'float64',
	      'order_date': 'datetime64[ns]',
      	'shipping_address' : 'object',
      	'payment_method' : 'object',
      	'status': 'object'

}
}
# --- Database setup ---
db_name = 'ecommerce.db'
conn = None  # Initialize connection to None

try:
    # Establish a connection to the SQLite database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    print(f"Database '{db_name}' created and connected successfully.")


    cursor.execute("DROP TABLE IF EXISTS customers;")
    cursor.execute("DROP TABLE IF EXISTS products;")
    cursor.execute("DROP TABLE IF EXISTS orders;")


    # Create tables
    cursor.execute(customers_schema)
    cursor.execute(products_schema)
    cursor.execute(orders_schema)
    print("Tables 'customers', 'products', and 'orders' created successfully.")

    # --- Load data from CSV files into the tables using pandas ---
    csv_to_table_map = {
        '/content/customers.csv': 'customers',
        '/content/products.csv': 'products',
        '/content/orders.csv': 'orders'
    }

    for csv_file, table_name in csv_to_table_map.items():
        if os.path.exists(csv_file):
            print(f"\nProcessing '{csv_file}' for table '{table_name}'...")

            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(csv_file)

            # 1. Get the expected schema for the current table
            expected_schema = COLUMN_DATA_TYPES[table_name]
            expected_cols = list(expected_schema.keys())

            # 2. Handle missing/extra columns
            # Drop columns from DataFrame that are not in the schema
            df = df[df.columns.intersection(expected_cols)]

            # Add any missing columns and fill with None (which becomes NULL in SQL)
            for col in expected_cols:
                if col not in df.columns:
                    df[col] = None

            # 3. Reorder columns to match the defined schema exactly
            df = df[expected_cols]

            # 4. Enforce data types
            for col, dtype in expected_schema.items():
                if 'datetime' in dtype:
                    # Use pd.to_datetime for date/time columns, coercing errors to NaT (Not a Time)
                    df[col] = pd.to_datetime(df[col], errors='coerce')
                else:
                    # Use astype for other columns, handling potential conversion errors
                    try:
                        df[col] = df[col].astype(dtype)
                    except (ValueError, TypeError) as e:
                        print(f"  - Warning: Could not convert column '{col}' to {dtype}. Error: {e}. Leaving as is.")


            # Use the to_sql method to insert the cleaned DataFrame
            df.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"  -> Data from '{csv_file}' loaded into '{table_name}' table successfully.")
        else:
            print(f"Warning: '{csv_file}' not found. Skipping data load for '{table_name}'.")

    # Commit the changes to the database
    conn.commit()
    print("\nData committed to the database successfully.")

except sqlite3.Error as e:
    print(f"Database error: {e}")
except pd.errors.EmptyDataError as e:
    print(f"Pandas error: {e}. One of the CSV files might be empty.")
except KeyError as e:
    print(f"Schema definition error: A column is missing from the TABLE_DATA_TYPES dictionary: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    # Close the connection if it was established
    if conn:
        conn.close()
        print("Database connection closed.")

Database 'ecommerce.db' created and connected successfully.
Tables 'customers', 'products', and 'orders' created successfully.

Processing '/content/customers.csv' for table 'customers'...
  -> Data from '/content/customers.csv' loaded into 'customers' table successfully.

Processing '/content/products.csv' for table 'products'...
  -> Data from '/content/products.csv' loaded into 'products' table successfully.

Processing '/content/orders.csv' for table 'orders'...
  -> Data from '/content/orders.csv' loaded into 'orders' table successfully.

Data committed to the database successfully.
Database connection closed.


In [8]:
!pip install google-genai

In [9]:
from google import genai
from google.colab import userdata

In [10]:
genai_client = genai.Client(api_key=userdata.get('Google_Api_key'))

In [11]:
prompt = """
### **ROLE**

You are an expert-level SQLite Database Engineer specializing in Natural Language to SQL (NL2SQL) translation. Your sole function is to convert user questions written in plain English into accurate, efficient, and syntactically correct SQLite queries based on a fixed database schema.

-----

### **CONTEXT**

You are the core translation engine for a business intelligence dashboard. This tool allows non-technical employees to query the company's e-commerce database using natural language. The database dialect is always **SQLite**. Your responses will be executed directly on the database.

The database consists of the following three tables:

**`customers` table:**

```sql
CREATE TABLE customers (
  customer_id INT PRIMARY KEY,
	first_name VARCHAR(50),
	last_name VARCHAR(50),
	email VARCHAR(50),
	phone_number VARCHAR(50),
	address VARCHAR(50),
	city VARCHAR(50),
	country VARCHAR(50),
	postal_code VARCHAR(50),
	membership_level VARCHAR(7)
);
```

**`products` table:**

```sql
CREATE TABLE products (
  product_id INT PRIMARY KEY,
	product_name VARCHAR(50),
	product_category VARCHAR(50),
	product_price DECIMAL(8,2),
	product_description TEXT,
	product_weight DECIMAL(4,1),
	product_color VARCHAR(5),
	product_brand VARCHAR(50),
	product_material VARCHAR(9),
	product_size VARCHAR(11),
	product_stock_quantity INT
);
```

**`orders` table:**

```sql
CREATE TABLE orders (
  order_id INT PRIMARY KEY,
	customer_id VARCHAR(50),
	product_id VARCHAR(50),
	quantity INT,
	unit_price DECIMAL(6,2),
	total_price DECIMAL(10,2),
	order_date DATE,
	shipping_address VARCHAR(50),
	payment_method VARCHAR(16),
	status VARCHAR(9),
  FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
  FOREIGN KEY (product_id) REFERENCES products(product_id)
);
```

-----

### **TASK**

Your task is to receive a user's question in natural language and convert it into a single, executable SQLite query. Follow these steps meticulously:

1.  **Analyze the User's Query:** Deconstruct the user's question to understand their core intent. Identify the specific data, conditions, aggregations (like `SUM`, `COUNT`, `AVG`), and ordering they are asking for.
2.  **Map to the Schema:** Map the entities from the user's query to the appropriate tables (`customers`, `products`, `orders`) and columns. Determine the necessary `JOIN` operations using `customers.customer_id` and `products.product_id` as foreign keys in the `orders` table.
3.  **Construct the SQLite Query:** Write a clean and efficient `SELECT` statement that is syntactically correct for SQLite. Ensure all table and column names are accurate.
4.  **Handle Ambiguity:** If the user's query is vague, ambiguous, or lacks the necessary information to create a precise query, do not guess. Instead, formulate a specific, targeted question to ask the user for the missing information.

-----

### **CONSTRAINTS**

  * **Read-Only Operations:** You must **ONLY** generate `SELECT` queries. Never generate `INSERT`, `UPDATE`, `DELETE`, `DROP`, or any other data-modifying statements.
  * **Adhere Strictly to Schema:** Only use the tables and columns defined in the context. Do not invent or assume the existence of any other tables or columns.
  * **No Explanations:** Do not add any conversational text or explanations about the query you generate. Your output must strictly follow the specified format.
  * **Single Query Only:** The final output must be a single, complete, and executable SQL query.
  * **Handle Impossibility:** If a request is impossible to fulfill with the given schema (e.g., "Which employee made the most sales?"), state clearly that the request cannot be completed and briefly explain why.

-----

### **EXAMPLES**

**Example 1: Simple Lookup**

  * **User Query:** "Show me all customers who live in Noida"
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT * FROM customers WHERE city = 'Noida';"
    }
    ```

**Example 2: Complex Join and Aggregation**

  * **User Query:** "What are the names of the top 3 products with the highest total revenue?"
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT T2.product_name FROM orders AS T1 INNER JOIN products AS T2 ON T1.product_id = T2.product_id GROUP BY T2.product_name ORDER BY SUM(T1.total_price) DESC LIMIT 3;"
    }
    ```

**Example 3: Ambiguous Query**

  * **User Query:** "Show me recent orders"
  * **Expected Output:**
    ```json
    {
      "status": "clarification_needed",
      "response": "Could you please define what 'recent' means? For example, 'in the last 7 days', 'this month', or 'since August 2025'."
    }
    ```

**Example 4: Impossible Query**

  * **User Query:** "Which warehouse has the most stock?"
  * **Expected Output:**
    ```json
    {
      "status": "error",
      "response": "I cannot answer this question as the database does not contain information about warehouses."
    }
    ```

-----

### **OUTPUT FORMAT**

Your final response must be a single JSON object with two keys:

1.  `"status"`: A string with one of three possible values: `"success"`, `"clarification_needed"`, or `"error"`.
2.  `"response"`:
      * If `status` is `"success"`, this will be a string containing the complete SQLite query.
      * If `status` is `"clarification_needed"`, this will be a string containing the clarifying question for the user.
      * If `status` is `"error"`, this will be a string explaining why the query could not be generated.
"""

In [12]:
import json
def get_sql_query(genai_client, prompt, user_query):


  contents = f"""
  {prompt}

  Here's the user query in english you need to work on:
  {user_query}
  """
  response = genai_client.models.generate_content(model='gemini-2.5-flash', contents=contents)
  # print(response)

  # Access the usage_metadata attribute
  usage_metadata = response.usage_metadata

  # Print the different token counts
  print(f"Input Token Count: {usage_metadata.prompt_token_count}")
  print(f"Thoughts Token Count: {response.usage_metadata.thoughts_token_count}")
  print(f"Output Token Count: {usage_metadata.candidates_token_count}")
  print(f"Total Token Count: {usage_metadata.total_token_count}")

  output = json.loads(response.text.replace('```json', '').replace('```', ''))

  return output


In [13]:
import sqlite3
import pandas as pd

def execute_query(query, db_name='ecommerce.db'):

    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Execute the query
        print(f"\nExecuting query on '{db_name}':\n{query}")
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Get column names from the cursor description
        columns = [description[0] for description in cursor.description]

        # Format results as a dataframe for easier use
        results_as_dict = [dict(zip(columns, row)) for row in results]
        results_df = pd.DataFrame(results_as_dict)

        print("Query executed successfully.")
        return results_df

    except sqlite3.Error as e:
        print(f"Database error executing query: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None
    finally:
        if conn:
            conn.close()

In [14]:
def text2sql(genai_client, prompt, user_query):
  output = get_sql_query(genai_client, prompt, user_query)
  if output['status'] == 'success':
    results = execute_query(output['response'])
    return results
  return output

In [15]:
text2sql(genai_client, prompt, "Show me the order count by country")

Input Token Count: 1481
Thoughts Token Count: 207
Output Token Count: 73
Total Token Count: 1761

Executing query on 'ecommerce.db':
SELECT T2.country, COUNT(T1.order_id) AS order_count FROM orders AS T1 INNER JOIN customers AS T2 ON T1.customer_id = T2.customer_id GROUP BY T2.country;
Query executed successfully.


,country,order_count
0,Afghanistan,7
1,Albania,3
2,Angola,3
3,Argentina,8
4,Armenia,6
...,...,...
109,Uzbekistan,2
110,Venezuela,8
111,Vietnam,8
112,Yemen,3


In [16]:
text2sql(genai_client, prompt, "Show me the order count by country")

Input Token Count: 1481
Thoughts Token Count: 230
Output Token Count: 69
Total Token Count: 1780

Executing query on 'ecommerce.db':
SELECT T2.country, COUNT(T1.order_id) FROM orders AS T1 INNER JOIN customers AS T2 ON T1.customer_id = T2.customer_id GROUP BY T2.country;
Query executed successfully.


,country,COUNT(T1.order_id)
0,Afghanistan,7
1,Albania,3
2,Angola,3
3,Argentina,8
4,Armenia,6
...,...,...
109,Uzbekistan,2
110,Venezuela,8
111,Vietnam,8
112,Yemen,3


In [17]:
text2sql(genai_client, prompt, "Give me the order count by day of month")

ServerError: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}

In [ ]:
text2sql(genai_client, prompt, "What are my most highest total quantity sold products")